In [17]:
import pandas as pd
import numpy as np
import lightgbm as lgb

In [2]:
path = '/home/kai/data/kaggle/talkingdata/data/'
train = pd.read_csv(path + 'train_cleaned_final.csv')
test = pd.read_csv(path + 'test_cleaned_final.csv')

train.drop(['is_attributed', 'day', 'timestamp'], axis=1, inplace = True)
test.drop(['day', 'timestamp'], axis=1, inplace = True)

In [5]:
test = test[list(train.columns)]
df_all = pd.concat([train,test])

train_length = len(train)
test_length = len(test)

import gc
del train
del test
gc.collect()

label = np.concatenate([np.zeros(train_length), np.ones(test_length)]) 
df_all['label'] = label

import sys
print(sys.getsizeof(df_all) / 1024**3)

26

# prepare train, val

In [18]:
from sklearn.model_selection import train_test_split
trainset, valset = train_test_split(df_all.sample(frac=1, random_state=233),test_size=0.3, random_state=42)

In [19]:
categorical_col = ['ip', 'app', 'device', 'os', 'channel']
feature_cols = ['ip', 'app', 'device', 'os', 'channel', 'hour', 'minute', 'second']

# Train LGB

In [ ]:
target = 'label'
y_train = trainset[target].values
y_val = valset[target].values

lgb_train = lgb.Dataset(trainset[feature_cols], y_train, categorical_feature = categorical_col)
lgb_val = lgb.Dataset(valset[feature_cols], y_val, categorical_feature = categorical_col)

zeros = len(y_train[y_train == 0])
# scale_pos_weight = len(y_train[y_train == 0]) / len(y_train[y_train == 1])
import gc
gc.collect()

354

In [ ]:
params = {
        'objective': 'binary',
        'boosting': 'gbdt',
        'num_rounds': 500,
        'learning_rate': 0.1,
        'num_leaves': 9,
        'num_threads': 20, # best speed: set to number of real cpu cores, which is vCPU/2
        'device': 'cpu',
        'max_depth': -1, # no limit. This is used to deal with over-fitting when #data is small.
        'min_data_in_leaf': 20,  #minimal number of data in one leaf. Can be used to deal with over-fitting
        'feature_fraction': 0.9, #For example, if set to 0.8, will select 80% features before training each tree.  speed up training / deal with over-fitting
        'feature_fraction_seed': 1,
        'early_stopping_round':30,
        'bagging_fraction': 0.9, #Randomly select part of data without resampling
        'bagging_freq': 1, #frequency for bagging, 0 means disable bagging. k means will perform bagging at every k iteration. to enable bagging, bagging_fraction should be set as well
        'bagging_seed': 1,
        #'max_bin': 255,
        'verbose': 0,
#         'scale_pos_weight': scale_pos_weight,
        'metric' : 'binary_logloss'
    }

model = lgb.train(params, train_set=lgb_train, valid_sets=lgb_val, verbose_eval=10)

/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:104: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1027: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:668: UserWarning: categorical_feature in param dict is overrided.
  warnings.warn('categorical_feature in param dict is overrided.')


Training until validation scores don't improve for 30 rounds.
[10]	valid_0's binary_logloss: 0.337234
[20]	valid_0's binary_logloss: 0.249118
[30]	valid_0's binary_logloss: 0.21775
[40]	valid_0's binary_logloss: 0.20609
[50]	valid_0's binary_logloss: 0.20056
[60]	valid_0's binary_logloss: 0.197744
[70]	valid_0's binary_logloss: 0.196125
[80]	valid_0's binary_logloss: 0.195042
[90]	valid_0's binary_logloss: 0.194081
[100]	valid_0's binary_logloss: 0.193317
[110]	valid_0's binary_logloss: 0.192665
[120]	valid_0's binary_logloss: 0.191849
[130]	valid_0's binary_logloss: 0.191469
[140]	valid_0's binary_logloss: 0.191171
[150]	valid_0's binary_logloss: 0.190881
[160]	valid_0's binary_logloss: 0.190444
[170]	valid_0's binary_logloss: 0.19
[180]	valid_0's binary_logloss: 0.189669
[190]	valid_0's binary_logloss: 0.189448
[200]	valid_0's binary_logloss: 0.18911
[210]	valid_0's binary_logloss: 0.188911
[220]	valid_0's binary_logloss: 0.188544
[230]	valid_0's binary_logloss: 0.188373
[240]	valid_

In [23]:
from sklearn.metrics import roc_auc_score
pred = model.predict(df_all[feature_cols])
df_all['pred'] = pred
print(roc_auc_score(df_all[target].values,pred))

NameError: name 'model' is not defined

In [ ]:
train = df_all.loc[:train_length]
train_final = train.sort_values(['pred'], ascending = False)

In [ ]:
thresh = 5000* 10000
index = list(train_final.index.values[:thresh])


In [ ]:
import json as js
file = '/home/kai/data/kaggle/talkingdata/wl/data/trainset/train_index.json'
with open(file, 'w') as myjs:
    js.dump(all_features, myjs)